# Проект по SQL

**Цель проета:**
Проанализировать базу данных и выявить данные, которые помогут сформулировать ценностное предложение для нового продукта.

## Создадим подключение к базе данных

In [1]:
import pandas as pd 
from sqlalchemy import create_engine # устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя  
             'pwd': 'Sdf4$2;d-d30pp', # пароль 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',  
             'port': 6432, # порт подключения  
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],  db_config['pwd'],  db_config['host'],  db_config['port'],  db_config['db']) # сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Выведем первые строки таблиц, изучим имеющиеся данные

In [2]:
#Функция для выводы первых 5 строк таблицы
def read_head (bd):
    query = '''
        SELECT * 
        FROM {} 
        
    '''.format(bd)
    rez = pd.io.sql.read_sql(query, con = engine) 
    display(rez.head(5))
   
rez = read_head('books')


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
rez1 = read_head('authors')

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
rez2 = read_head('ratings')

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
rez3 = read_head('reviews')

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [6]:
rez4 = read_head('publishers')

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


## Создадим запросы для ответов на поставленные вопросы


### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query1 = '''
    SELECT COUNT(book_id)
    FROM books as b
    WHERE b.publication_date::date > '2000-01-01'
        
    '''
books_tot = pd.io.sql.read_sql(query1, con = engine) 
display('Количество книг, выпущенных после 01.01.2000 г.',books_tot)
 

'Количество книг, выпущенных после 01.01.2000 г.'

,count
0,819


**После 01.01.2000г. выпущено 819 книг**

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query2 = '''
    SELECT DISTINCT  b.title,b.book_id, 
           COUNT (DISTINCT rew.review_id) rew,
           ROUND(AVG (rat.rating),2)
    FROM books b  LEFT JOIN reviews rew ON b.book_id = rew.book_id
    JOIN ratings rat ON b.book_id = rat.book_id
    GROUP BY b.title,b.book_id
    ORDER BY rew DESC, ROUND(AVG (rat.rating),2) DESC 
    '''
books_r = pd.io.sql.read_sql(query2, con = engine) 
books_r.columns = ['Наименование',' book_id','Количество обзоров','Средняя оценка']
display('Количество обзоров и средняя оценка книг',books_r)

'Количество обзоров и средняя оценка книг'

,Наименование,book_id,Количество обзоров,Средняя оценка
0,Twilight (Twilight #1),948,7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,302,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,299,6,4.29
3,The Book Thief,656,6,4.26
4,The Glass Castle,734,6,4.21
...,...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,191,0,4.00
996,Essential Tales and Poems,221,0,4.00
997,Leonardo's Notebooks,387,0,4.00
998,Anne Rice's The Vampire Lestat: A Graphic Novel,83,0,3.67


**Больше всего обзоров  вышло на "Сумерки", но средняя оценка у них ниже, чем у разных частей Грри Поттера со довольно высокой среденей оценкой (4.41 и 4.29) и с 6 обзорами на каждую часть. Всего ест данные о 1000 книгах.**

### Исключим из анализа брошюры: определим издательство, которое выпустило наибольшее число книг толще 50 страниц. 

In [10]:
query3 = '''
     WITH pub AS (SELECT b.publisher_id, COUNT(b.book_id), b.num_pages
    FROM books b
    WHERE b.num_pages > 50
    GROUP BY  b.publisher_id, b.num_pages
    ORDER BY COUNT(b.book_id) DESC
    LIMIT 1) 
    SELECT p.publisher
    FROM publishers p join pub  ON pub.publisher_id = p.publisher_id
    '''
publ_r = pd.io.sql.read_sql(query3, con = engine) 
display ('Издательство', publ_r)

'Издательство'

,publisher
0,Penguin Books


**Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books"**

### Определим автора с самой высокой средней оценкой книг. Будем учитывать только книги с 50 и более оценками.

In [11]:
query4 = '''
    WITH b_avg AS (SELECT   b.author_id,rat.book_id, COUNT(rat.rating_id),AVG(rat.rating)
    FROM ratings rat join books b on rat.book_id = b.book_id
    GROUP BY b.author_id,rat.book_id
    HAVING COUNT(rat.rating_id) >=50
    ORDER BY AVG (rat.rating) DESC)
    
    SELECT a.author
    FROM authors a join 
    (SELECT b_avg.author_id, AVG (b_avg.avg) auth
    FROM b_avg
    GROUP BY   b_avg.author_id 
    ORDER BY auth DESC
    LIMIT 1) auth ON  a.author_id = auth.author_id
    
    '''
auth = pd.io.sql.read_sql(query4, con = engine) 
display ('Автор книг с самой высокой средней оценкой', auth)

'Автор книг с самой высокой средней оценкой'

,author
0,J.K. Rowling/Mary GrandPré


**Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками -Джоан Роулинг с книгами, иллюстрированными Мэри Грандпре**

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query5 = '''WITH us AS (SELECT  DISTINCT rat.username as name, COUNT (rat.rating_id) as count
    FROM ratings rat
    GROUP BY name
    HAVING COUNT (rat.rating_id)  > 50
    ORDER BY count DESC)
    
    SELECT ROUND (AVG(rew.count),0)
    FROM (SELECT DISTINCT rew.username, COUNT (rew.review_id )
    FROM us LEFT JOIN reviews rew ON us.name = rew.username
    GROUP BY rew.username)rew
    '''
rew = pd.io.sql.read_sql(query5, con = engine) 
display ('среднее количество обзоров от пользователей, которые поставили больше 50 оценок', rew)

'среднее количество обзоров от пользователей, которые поставили больше 50 оценок'

,round
0,24.0


**Пользователей, которые поставили больше 50 оценок, в среднем сделали 24 обзора**

**Вывод**

*После подключения к базе и изучения состава данных, мы составили запросы и  получили слежующие ответы на поставленные вопросы:*

- После 01.01.2000г. выпущено 819 книг.

- В Базе данных есть данные о рейтинге и обзорах гна 1000 книг, среди которых больше всего обзоров вышло на "Сумерки",но средняя оценка у них ниже, чем у разных частей Грри Поттера, котороые собрали по 6 обзоров, и имеют довольно высокую среденюю оценку (4.41 и 4.29). 

- Издательство, которое выпустило наибольшее число книг толще 50 страниц - "Penguin Books"

- Автор с самой высокой средней оценкой книг среди книг с 50 и более оценками -Джоан Роулинг с книгами, иллюстрированными Мэри Грандпре
- Пользователей, которые поставили больше 50 оценок, в среднем сделали 24 обзора
